In [1]:
import sys;
sys.path.insert(0, '/home/patrick/projects/omegaml')
import omegaml as om
om.defaults.OMEGA_USERID

'testing3'

In [ ]:
om.datasets.list()

In [4]:
import pandas as pd

df = pd.DataFrame({'x': range(10)})
om.datasets.put(df, 'sample')

<Metadata: Metadata(attributes={},gridfile=<GridFSProxy: (no file)>,objid=None,created=2018-01-16 20:07:33.481106,bucket=store,s3file={},collection=store.data_.sample.datastore,name=sample,id=5a5e5b85de39d13b8b0a42a6,kind=pandas.dfrows,uri=None,kind_meta={'columns': {'_idx#0_0': '_idx#0_0', 'x': 'x'}, 'idx_meta': {'names': [None]}, 'dtypes': {'_idx#0_0': 'int64', 'x': 'int64'}},prefix=data/)>

In [3]:
print(om.datasets.get('dfx', lazy=True).loc[2:5].value.drop_duplicates())

   x  y
2  2  2
3  3  3
4  4  4
5  5  5


In [50]:
om.datasets.get('dfx', lazy=True).query(x__gt=2, x__lt=5).inspect()

{'explain': 'specify explain=True',
 'projection': ['x', 'y'],
 'query': {'$and': [{'x': {'$lt': 5}}, {'x': {'$gt': 2}}]}}

In [51]:
om.datasets.get('dfx', lazy=True).query(x__gt=2, x__lt=5).inspect(explain=True)

{'explain': {'executionStats': {'allPlansExecution': [],
   'executionStages': {'advanced': 4,
    'executionTimeMillisEstimate': 0,
    'inputStage': {'advanced': 4,
     'direction': 'forward',
     'docsExamined': 1100,
     'executionTimeMillisEstimate': 0,
     'filter': {'$and': [{'x': {'$lt': 5}}, {'x': {'$gt': 2}}]},
     'invalidates': 0,
     'isEOF': 1,
     'nReturned': 4,
     'needTime': 1097,
     'needYield': 0,
     'restoreState': 8,
     'saveState': 8,
     'stage': 'COLLSCAN',
     'works': 1102},
    'invalidates': 0,
    'isEOF': 1,
    'nReturned': 4,
    'needTime': 1097,
    'needYield': 0,
    'restoreState': 8,
    'saveState': 8,
    'stage': 'PROJECTION',
    'transformBy': {'_idx#0_0': 1, 'x': 1, 'y': 1},
    'works': 1102},
   'executionSuccess': True,
   'executionTimeMillis': 1,
   'nReturned': 4,
   'totalDocsExamined': 1100,
   'totalKeysExamined': 0},
  'ok': 1.0,
  'queryPlanner': {'indexFilterSet': False,
   'namespace': 'testing3.omegaml.data_.df

In [64]:
print(om.datasets.get('dfx', lazy=True).sort(['-x', '+y']).head(5).value)

       x    y
999  999  999
998  998  998
997  997  997
996  996  996
995  995  995


In [70]:
print(om.datasets.get('dfx', lazy=True).skip(5).head(3).value)

   x  y
5  5  5
6  6  6
7  7  7


In [84]:
mdf = om.datasets.get('dfx', lazy=True)
print(id(mdf))
print(id(mdf.query(x__gt=2, x__lt=5)))
print(id(mdf.query_inplace(x__gt=2, x__lt=5)))
print(mdf.value)

140341971523792
140341971587648
140341971523792
   x  y
3  3  3
4  4  4
3  3  3
4  4  4


In [99]:
import pandas as pd
dfl = pd.DataFrame({'x': range(3)})
dfr = pd.DataFrame({'x': range(3), 'y': range(3)})
om.datasets.put(dfl, 'dfxl', append=False)
om.datasets.put(dfr, 'dfxr', append=False)
mdfl = om.datasets.get('dfxl', lazy=True)
mdfr = om.datasets.get('dfxr', lazy=True)
print(mdfl.merge(mdfr, on='x').value)

   x  y
0  0  0
1  1  1
2  2  2


In [111]:
mdf = om.datasets.getl('dfx')
print(mdf.groupby('x').x.mean().head(5))

NameError: name 'x' is not defined

In [120]:
mdf = om.datasets.getl('dfx')
print(mdf.groupby('x').agg(dict(x='sum', y='mean')).head(5))

   x_sum  y_mean
x               
0      0     0.0
1      2     1.0
2      4     2.0
3      6     3.0
4      8     4.0


In [122]:
mdf = om.datasets.getl('dfx')
print(mdf.groupby('x').agg(dict(x='std', y='avg')).head(5))

   x_std  y_avg
x              
0    0.0    0.0
1    0.0    1.0
2    0.0    2.0
3    0.0    3.0
4    0.0    4.0


In [125]:
mdf = om.datasets.getl('dfx')
print(mdf.groupby('x').agg(dict(x='first', y='avg')).head(5))

   x_first  y_avg
x                
0        0    0.0
1        1    1.0
2        2    2.0
3        3    3.0
4        4    4.0


In [143]:
from omegaml.store.queryops import GeoJSON
locations = [{'location': {'coordinates': [-74.0059413, 40.7127837],
                           'type': 'Point'},
              'place': 'New York'},
             {'location': {'coordinates': [6.1431577, 46.2043907],
                           'type': 'Point'},
              'place': 'Geneva'},
             {'location': {'coordinates': [7.4474468, 46.9479739],
                           'type': 'Point'},
              'place': 'Bern'},
             {'location': {'coordinates': [8.541694, 47.3768866],
                           'type': 'Point'},
              'place': 'Zurich'}]
geodf = pd.DataFrame(locations)
geodf['location'] = geodf.location.apply(lambda v: GeoJSON(v))
om.datasets.put(geodf, 'geosample', append=False, index='@location')
r = om.datasets.getl('geosample', location__near=dict(location=(7.4474468, 46.9479739))).value
print(r['place'])

2        Bern
3      Zurich
1      Geneva
0    New York
Name: place, dtype: object


In [153]:
from sklearn.linear_model import LinearRegression

df = pd.DataFrame(dict(x=range(10), y=range(20,30)))
clf = LinearRegression()
clf.fit(df[['x']], df[['y']])
clf.predict(df[['x']])

om.models.put(clf, 'lrmodel')


<Metadata: Metadata(s3file={},kind_meta={},created=2018-01-13 18:21:11.008630,objid=None,id=5a5a4e17de39d176ba6b986c,uri=None,gridfile=<GridFSProxy: omegaml.models_.lrmodel.omm>,prefix=models/,kind=sklearn.joblib,bucket=omegaml,attributes={},name=lrmodel,collection=None)>

In [159]:
om.models.get('lrmodel')

result = om.runtime.model('lrmodel').predict(df[['x']])
result.get()


array([[ 20.],
       [ 21.],
       [ 22.],
       [ 23.],
       [ 24.],
       [ 25.],
       [ 26.],
       [ 27.],
       [ 28.],
       [ 29.]])

In [ ]:
import sys;
sys.path.insert(0, '/home/patrick/projects/omegaml')


import omegaml as om 
import pandas as pd 

om.models.get('lrmodel')

df = pd.DataFrame(dict(x=range(70,80)))
om.datasets.put(df, 'testlrmodel')

result = om.runtime.model('lrmodel').predict('testlrmodel')
result.get()



In [7]:
import sys;
sys.path.insert(0, '/home/patrick/projects/omegaml')


import omegaml as om 
import pandas as pd 
om.runtime.settings()

KeyboardInterrupt: 

In [5]:
om.models.mongodb


Database(MongoClient(host=['dokku.me:27017'], document_class=dict, tz_aware=False, connect=False, read_preference=Primary(), serverselectiontimeoutms=2500), 'testing3')

In [161]:
om.models.get('lrmodel')

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [5]:
import pandas as pd

clf = om.models.get('lrmodel')
df = pd.DataFrame(dict(x=range(70,80)))
clf.predict(df[['x']])

array([[ 90.],
       [ 91.],
       [ 92.],
       [ 93.],
       [ 94.],
       [ 95.],
       [ 96.],
       [ 97.],
       [ 98.],
       [ 99.]])